In [16]:
import numpy as np
import pandas as pd
import pickle
from collections import Counter
from sklearn import svm, cross_validation, neighbors
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from collections import Counter

In [17]:
def process_data_for_labels(ticker):
    hm_days = 7
    df = pd.read_csv('sp500_joined_closes.csv')
    tickers = df.columns.values.tolist() # get the values of the colums on lis## values of the colums = titles of thew colunms
    df.fillna(0, inplace=True)
    
    for i in range(1, hm_days+1):
        df['{}_{}d'.format(ticker, i)] = (df[ticker].shift(-i) - df[ticker]) / df[ticker]
        
    df.fillna(0, inplace=True)
    #print(df)
    #print(df.columns.values.tolist())
    return tickers, df

In [52]:
def buy_sell_hold(*args):
    #the function returns the labels
    
    # the args cotain the data in tuples, each tuple has the 7 days of a price of a date
    
    cols = [c for c in args] # cols contains each row of 7 days of a price date
    requirement = 0.028
    for col in cols:
        if col > requirement:
            return 1
        if col < -requirement:
            return -1
    return 0
    

In [53]:
def extract_featuresets(ticker):
    tickers, df = process_data_for_labels(ticker)
    df['{}_target'.format(ticker)] = list(map( buy_sell_hold,
                                               df['{}_1d'.format(ticker)],
                                               df['{}_2d'.format(ticker)],
                                               df['{}_3d'.format(ticker)],
                                               df['{}_4d'.format(ticker)],
                                               df['{}_5d'.format(ticker)],
                                               df['{}_6d'.format(ticker)],
                                               df['{}_7d'.format(ticker)] ))
    
    vals = df['{}_target'.format(ticker)].values.tolist() # list of the values of labels column
    str_vals = [str(i) for i in vals]
    #print(str_vals)
    print('Data spread:',Counter(str_vals))
    
    
    df.fillna(0, inplace=True)
    df = df.replace([np.inf, -np.inf], np.nan)
    df.dropna(inplace=True)
    
    
    df_vals = df[[ticker for ticker in tickers]].set_index('date').pct_change()# the percent change between rows
    df_vals = df_vals.replace([np.inf, -np.inf], 0)
    df_vals.fillna(0, inplace=True)
    
    X = df_vals.values # features
    y = df['{}_target'.format(ticker)].values # labels
    
    return X,y,df

In [54]:
def do_ml(ticker):
    X, y, df = extract_featuresets(ticker)
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.25)
    
    #clf = neighbors.KNeighborsClassifier()
    
    clf = VotingClassifier([('lsvc',svm.LinearSVC()),
                            ('knn',neighbors.KNeighborsClassifier()),
                            ('rfor',RandomForestClassifier())])
    
    clf.fit(X_train, y_train)
    
    
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print("predicted spread:", Counter(predictions))
    
    return confidence

In [55]:
do_ml("BAC")

Data spread: Counter({'0': 490, '1': 428, '-1': 341})


/home/dan/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy: 0.37777777777777777
predicted spread: Counter({0: 168, -1: 106, 1: 41})


/home/dan/.local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.37777777777777777

In [7]:
df["XOM"].head(10)

NameError: name 'df' is not defined

In [ ]:
print(len([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, -1, -1, -1,]))